# BERT 한국어 Tokenizing (Morphology) End to End

In [1]:
# import library
import collections, re, unicodedata, six, csv, os, copy, json, math
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
class InputExample:
    """Input의 고유 id, text A and B, label을 기록하는 객체"""
    def __init__(self, guid, text_a, text_b=None, label=None):
        self.guid = guid
        self.text_a = text_a
        self.text_b = text_b
        self.label = label

class InputFeatures(object):
    """Input의 embedding feature 및 label, real sample 여부를 기록하는 객체"""
    def __init__(self,
               input_ids,
               input_mask,
               segment_ids,
               label_id,
               is_real_example=True):
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids
        self.label_id = label_id
        self.is_real_example = is_real_example
        
class DataProcessor(object):
    """Processor 모체 클래스"""
    @classmethod
    def _read_tsv(cls, input_file, quotechar=None):
        """tab 구분자로 구분된 파일을 읽는 메서드"""
        with tf.gfile.Open(input_file, "r") as f:
            reader = csv.reader(f, delimiter="\t", quotechar=quotechar)
            lines = []
            for line in reader:
                lines.append(line)
            return lines
        
class SmishingProcessor(DataProcessor):
    """DataProcessor를 상속받아 InputExample 객체의 list를 생성하는 객체"""
    def get_train_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "train.tsv")), "train")

    def get_dev_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "dev.tsv")), "dev")

    def get_test_examples(self, data_dir):
        return self._create_examples(
            self._read_tsv(os.path.join(data_dir, "test.tsv")), "test")

    def get_labels(self):
        return ["0", "1"]

    def _create_examples(self, lines, set_type):
        examples = []
        for (i, line) in enumerate(lines):
            if i == 0:
                continue
            guid = "%s-%s" % (set_type, i)
            text_a = convert_to_unicode(line[2])
            if set_type == "test":
                label = "0"
            else:
                label = convert_to_unicode(line[-1])
            examples.append(
                InputExample(guid=guid, text_a=text_a, label=label))
        return examples
    
def convert_to_unicode(text):
    if six.PY3: # Python3일 경우 여기
        if isinstance(text, str):
            # string일 경우 그대로 반환
            return text
        elif isinstance(text, bytes):
            # bytes일 경우 utf-8로 decoding
            return text.decode("utf-8", "ignore")
        else:
            # 이 외의 타입은 지원하지 않는다.
            raise ValueError("Unsupported string type: %s" % (type(text)))
    elif six.PY2: # Python2일 경우 여기
        if isinstance(text, str):
            # string일 경우 utf-8로 decoding
            return text.decode("utf-8", "ignore")
        elif isinstance(text, unicode):
            # unicode 타입일 경우 그대로 반환
            return text
        else:
            # 이 외의 타입은 지원하지 않는다.
            raise ValueError("Unsupported string type: %s" % (type(text)))
    else: # Python2, Python3 둘 다 아닐 경우 raise ValueError.
        raise ValueError("Not running on Python2 or Python 3?")
        
class PaddingInputExample(object):
    """Fake example so the num input examples is a multiple of the batch size.
    When running eval/predict on the TPU, we need to pad the number of examples
    to be a multiple of the batch size, because the TPU requires a fixed batch
    size. The alternative is to drop the last batch, which is bad because it means
    the entire output data won't be generated.
    We use this class instead of `None` because treating `None` as padding
    battches could cause silent errors.
    """

In [3]:
# parameter 미리 setting
flags = tf.flags
FLAGS = flags.FLAGS

# Jupyter notebook에서 사용시에만 필요
flags.DEFINE_string('f', '', 'kernel')

## Required parameters
flags.DEFINE_string(
    "data_dir", None,
    "The input data dir. Should contain the .tsv files (or other data files) "
    "for the task.")

flags.DEFINE_string(
    "bert_config_file", None,
    "The config json file corresponding to the pre-trained BERT model. "
    "This specifies the model architecture.")

flags.DEFINE_string("task_name", None, "The name of the task to train.")

flags.DEFINE_string("vocab_file", None,
                    "The vocabulary file that the BERT model was trained on.")

flags.DEFINE_string(
    "output_dir", None,
    "The output directory where the model checkpoints will be written.")

## Other parameters

flags.DEFINE_string(
    "init_checkpoint", None,
    "Initial checkpoint (usually from a pre-trained BERT model).")

flags.DEFINE_bool(
    "do_lower_case", True,
    "Whether to lower case the input text. Should be True for uncased "
    "models and False for cased models.")

flags.DEFINE_integer(
    "max_seq_length", 128,
    "The maximum total input sequence length after WordPiece tokenization. "
    "Sequences longer than this will be truncated, and sequences shorter "
    "than this will be padded.")

flags.DEFINE_bool("do_train", False, "Whether to run training.")

flags.DEFINE_bool("do_eval", False, "Whether to run eval on the dev set.")

flags.DEFINE_bool(
    "do_predict", False,
    "Whether to run the model in inference mode on the test set.")

flags.DEFINE_integer("train_batch_size", 32, "Total batch size for training.")

flags.DEFINE_integer("eval_batch_size", 8, "Total batch size for eval.")

flags.DEFINE_integer("predict_batch_size", 8, "Total batch size for predict.")

flags.DEFINE_float("learning_rate", 5e-5, "The initial learning rate for Adam.")

flags.DEFINE_float("num_train_epochs", 3.0,
                   "Total number of training epochs to perform.")

flags.DEFINE_float(
    "warmup_proportion", 0.1,
    "Proportion of training to perform linear learning rate warmup for. "
    "E.g., 0.1 = 10% of training.")

flags.DEFINE_integer("save_checkpoints_steps", 1000,
                     "How often to save the model checkpoint.")

flags.DEFINE_integer("iterations_per_loop", 1000,
                     "How many steps to make in each estimator call.")

flags.DEFINE_bool("use_tpu", False, "Whether to use TPU or GPU/CPU.")

tf.flags.DEFINE_string(
    "tpu_name", None,
    "The Cloud TPU to use for training. This should be either the name "
    "used when creating the Cloud TPU, or a grpc://ip.address.of.tpu:8470 "
    "url.")

tf.flags.DEFINE_string(
    "tpu_zone", None,
    "[Optional] GCE zone where the Cloud TPU is located in. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")

tf.flags.DEFINE_string(
    "gcp_project", None,
    "[Optional] Project name for the Cloud TPU-enabled project. If not "
    "specified, we will attempt to automatically detect the GCE project from "
    "metadata.")

tf.flags.DEFINE_string("master", None, "[Optional] TensorFlow master URL.")

flags.DEFINE_integer(
    "num_tpu_cores", 8,
    "Only used if `use_tpu` is True. Total number of TPU cores to use.")

In [4]:
dacon_path = '../dacon문자스미싱/filedown (2)/'
train_file = os.path.join('./output_dir/smishing/', 'train.tf_record')
train_file

'./output_dir/smishing/train.tf_record'

In [5]:
processor = SmishingProcessor()
label_list = processor.get_labels()

# get train samples
train_examples = processor.get_train_examples(dacon_path)
num_train_steps = int(
    len(train_examples) / FLAGS.train_batch_size * FLAGS.num_train_epochs)
num_warmup_steps = int(num_train_steps * FLAGS.warmup_proportion)

In [9]:
train_examples[5].text_a

'(XXX고객님 안녕하세요 유익한 어플 세가지 소개해드리고자 합니다.1. 스타알림 : 6개의 비밀번호 만으로 거래내역조회 가능 입출금통지가 무료 (창구에서는 한달에 900원 or 한건에 20원)2. 리브 : USJPEU 환전우대 90% 번호표 미리 뽑기 지인에게 로그인없이 바로 이체 창구에서 통장없이 바로 출금송금3. 리브메이트 : 잠자고 있는 카드포인트리 조회  계좌로 현금화 송금어플 설치시 직원번호 XXX 꼭 넣어주세요 .언제나 XXX은행을 이용해 주셔서 감사합니다.XXX은행판교종합금융센터XXX올림XXX-XXX-XXX무료수신거부XXX-XXX-XXX'

In [ ]:
import sentencepiece as spm



In [6]:
# fn: file_based_convert_examples_to_features

## Arguments
examples = train_examples
label_list = label_list
max_seq_length = FLAGS.max_seq_length
# tokenizer = FullTokenizer() # 예시로 tokenizing을 어떻게 하는지 전부 기록
output_file = train_file

In [7]:
writer = tf.python_io.TFRecordWriter(output_file)

In [8]:
ex_index = 5
example = examples[ex_index]

In [9]:
# fn: convert_single_example
isinstance(example, PaddingInputExample)

False

In [10]:
label_map = {}
for (i, label) in enumerate(label_list):
    label_map[label] = i
label_map

{'0': 0, '1': 1}

In [11]:
# class: FullTokenizer
path = '../KorBERT/2_bert_download_002_bert_morp_tensorflow/002_bert_morp_tensorflow/'
FLAGS.vocab_file = path + 'vocab.korean_morp.list' 
vocab_file = FLAGS.vocab_file
do_lower_case = FLAGS.do_lower_case
text = example.text_a
text

'(/SSO XXX/SL 고객/NNG 님/XSN 안녕/NNG 하/XSV 세요/EP+EF 유익/XR 한/XSA+ETM 어/IC 플/NNG 세/MM 가지/NNBC 소개/NNG 해/XSV+EC 드리/VX 고자/EC 합니다/VX+EF ./SF 1/SN ./SF 스타/NNG 알림/VV+ETN :/SC 6/SN 개/NNBC 의/JKG 비밀/NNG 번호/NNG 만/JX 으로/JKB 거래/NNG 내역/NNG 조회/NNG 가능/NNG 입출금/NNG 통지/NNG 가/JKS 무료/NNG (/SSO 창구/NNG 에서/JKB 는/JX 한/MM 달/NNG 에/JKB 900/SN 원/NNBC or/SL 한/MM 건/NNBC 에/JKB 20/SN 원/NNBC )/SSC 2/SN ./SF 리브/NNG :/SC USJPEU/SL 환전/NNG 우대/NNG 90/SN %/SY 번호표/NNG 미리/MAG 뽑/VV 기/ETN 지인/NNG 에게/JKB 로그인/NNG 없이/MAG 바로/MAG 이체/NNG 창구/NNG 에서/JKB 통장/NNG 없이/MAG 바로/MAG 출금/NNG 송금/NNG 3/SN ./SF 리브/NNG 메이트/NNG :/SC 잠자/VV 고/EC 있/VX 는/ETM 카드/NNG 포/NNG 인/VCP+ETM 트리/NNG 조회/NNG 계좌/NNG 로/JKB 현금화/NNG 송금/NNG 어/IC 플/NNG 설치/VV 시/EP 직원/NNG 번호/NNG XXX/SL 꼭/MAG 넣/VV 어/EC 주/VX 세요/EP+EF ./SF 언제나/MAG XXX/SL 은행/NNG 을/JKO 이용/NNG 해/XSV+EC 주/VX 셔서/EP+EC 감사/NNG 합니다/XSV+EF ./SF XXX/SL 은행/NNG 판교/NNG 종합/NNG 금융/NNG 센터/NNG XXX/SL 올림/VV+EC XXX/SL -/SY XXX/SL -/SY XXX/SL 무료/NNG 수신/NNG 거부/NNG XXX/SL -/SY XXX/SL -/SY XXX/SL'

In [12]:
def load_vocab(vocab_file):
    vocab = collections.OrderedDict()
    index = 0
    with tf.gfile.GFile(vocab_file, "r") as reader:
        while True:
            token = convert_to_unicode(reader.readline())
            if not token:
                break

            ### joonho.lim @ 2019-03-15
            if token.find('n_iters=') == 0 or token.find('max_length=') == 0 :
                continue
            token = token.split('\t')[0]

            token = token.strip()
            vocab[token] = index
            index += 1
    return vocab

vocab = load_vocab(vocab_file)
inv_vocab = {v:k for k, v in vocab.items()}

In [13]:
chr(0), chr(0xfffd)

('\x00', '�')

In [14]:
def _clean_text(text):
    output = [] # char을 저장할 list 생성
    for char in text:
        # 텍스트에서 Char 단위로 출력
        cp = ord(char)
        if cp == 0 or cp == 0xfffd or _is_control(char):
            # \x00이거나 �이거나 unicode cat.이 C로 시작할 경우
            # (개행문자 제외) output에 추가하지 않는다.
            continue
        if _is_whitespace(char):
            # 공백일 경우 " "으로 output에 추가
            output.append(" ")
        else:
            # 이 외의 경우 전부 output에 추가
            output.append(char)
    # cleaning 작업을 거친 Text를 후처리하여 반환
    return "".join(output)

# char 단위 함수들
def _is_whitespace(char):
    if char == " " or char == '\t' or char == '\n' or char == '\r':
        # 개행문자이거나 띄어쓰기면 True 반환
        return True
    cat = unicodedata.category(char)
    if cat == 'Zs':
        # unicode category가 Space Seperator면 True 반환
        return True
    # 이 외의 경우 전부 False 반환
    return False

def _is_control(char):
    if char == "\t" or char == "\n" or char == "\r":
        # 개행문자이면 False 반환
        return False
    cat = unicodedata.category(char)
    if cat.startswith("C"):
        # unicode category가
        # Cc(Control) 
        # Cf(format)
        # Co(Private Use, is 0)
        # Cs(Surrrogate, is 0)일 경우, True 반환
        return True
    # 이 외의 경우 전부 False 반환
    return False

def _is_punctuation(char):
    # 한국어 형태소 분석기이기 때문에 공백과 같은지 여부만 반환
    return char == ' '

In [15]:
text.strip()

'(/SSO XXX/SL 고객/NNG 님/XSN 안녕/NNG 하/XSV 세요/EP+EF 유익/XR 한/XSA+ETM 어/IC 플/NNG 세/MM 가지/NNBC 소개/NNG 해/XSV+EC 드리/VX 고자/EC 합니다/VX+EF ./SF 1/SN ./SF 스타/NNG 알림/VV+ETN :/SC 6/SN 개/NNBC 의/JKG 비밀/NNG 번호/NNG 만/JX 으로/JKB 거래/NNG 내역/NNG 조회/NNG 가능/NNG 입출금/NNG 통지/NNG 가/JKS 무료/NNG (/SSO 창구/NNG 에서/JKB 는/JX 한/MM 달/NNG 에/JKB 900/SN 원/NNBC or/SL 한/MM 건/NNBC 에/JKB 20/SN 원/NNBC )/SSC 2/SN ./SF 리브/NNG :/SC USJPEU/SL 환전/NNG 우대/NNG 90/SN %/SY 번호표/NNG 미리/MAG 뽑/VV 기/ETN 지인/NNG 에게/JKB 로그인/NNG 없이/MAG 바로/MAG 이체/NNG 창구/NNG 에서/JKB 통장/NNG 없이/MAG 바로/MAG 출금/NNG 송금/NNG 3/SN ./SF 리브/NNG 메이트/NNG :/SC 잠자/VV 고/EC 있/VX 는/ETM 카드/NNG 포/NNG 인/VCP+ETM 트리/NNG 조회/NNG 계좌/NNG 로/JKB 현금화/NNG 송금/NNG 어/IC 플/NNG 설치/VV 시/EP 직원/NNG 번호/NNG XXX/SL 꼭/MAG 넣/VV 어/EC 주/VX 세요/EP+EF ./SF 언제나/MAG XXX/SL 은행/NNG 을/JKO 이용/NNG 해/XSV+EC 주/VX 셔서/EP+EC 감사/NNG 합니다/XSV+EF ./SF XXX/SL 은행/NNG 판교/NNG 종합/NNG 금융/NNG 센터/NNG XXX/SL 올림/VV+EC XXX/SL -/SY XXX/SL -/SY XXX/SL 무료/NNG 수신/NNG 거부/NNG XXX/SL -/SY XXX/SL -/SY XXX/SL'

In [16]:
for i in range(20):
    if i % (20 // 4) == 0:
        print(i)

0
5
10
15


In [17]:
def print_c(text, is_print):
    if is_print:
        print(text)
    else:
        print(end='')

In [19]:
# FullTokenizer.tokenize(); End2End Tokenizer
text = example.text_a # text 초기화
print('************** START TOKENING MORPHLOGY **************')
split_tokens = [] 
# BasicTokenizer.tokenize()
print('\nOrigin Text:  ', text)
text = convert_to_unicode(text)
text = _clean_text(text)
print('\nCleaned Text: ', text)
# fn: whitespace_tokenize()
text = text.strip()
if not text: orig_tokens = []
else: orig_tokens = text.split()
print('\nOrig. Tokens: ', orig_tokens, end='\n\n')
split_tokens = []
for (ix, token) in enumerate(orig_tokens):
    if ix % (len(orig_tokens) // 5) == 0:
        # 전체 token 중 10개만 tokenizing 결과 출력
        is_print = True
    else:
        # 아닐 경우 출력하지 않는다.
        is_print = False
    print_c('orig Token : '+token, is_print=is_print)
    if do_lower_case: # True
        token = token.lower()
        # fn: _run_strip_accents
        t = unicodedata.normalize("NFD", token)
        # https://gist.github.com/Pusnow/aa865fa21f9557fa58d691a8b79f8a6d
        # 모든 음절을 정준 분해(Canonical Decomposition)시킴
        # '각'을 'ㄱ+ㅏ+ㄱ'으로 저장(출력되는 값은 동일)
        output = []
        for char in t:
            cat = unicodedata.category(char)
            if cat == "Mn":
                # unicode category가 "Mark, Nonspacing"일 경우 pass
                continue
            output.append(char)
        token = "".join(output)
        print_c('\tstripped accent+norm(NFD) Token : '+t, is_print=is_print)
    # fn: _run_split_on_punc()
    chars = list(token)
    i, start_new_word = 0, True
    output = []
    print_c('\tchars : '+str(chars), is_print)
    while i < len(chars):
        char = chars[i]
        if _is_punctuation(char): # == " "과 같은 표현
            output.append([char])
            start_new_word = True
        else:
            if start_new_word:
                output.append([])
            start_new_word = False
            output[-1].append(char)
        i += 1
        print_c('\t\t' + str(output), is_print=is_print)
    token = ["".join(x) for x in output]
    print_c('\t' + str(token), is_print=is_print)
    split_tokens.extend(token)
print('\nsplit_tokens : ', split_tokens)
# fn: whitespace_tokenize
print('\n_____WHITE_SPACING_____')
split_tokens = (' '.join(split_tokens)).strip()
if not split_tokens: output_tokens = []
else: output_tokens = split_tokens.split()
print('\noutput Tokens: ', output_tokens, end='\n\n')

split_tokens = [] # 최종 결과물 저장할 리스트
for tokens in output_tokens:
    tokens += '_' # adding '_'
    # WordpieceTokenizer.tokenize()
    unk_token = "[UNK]"
    max_input_chars_per_word = 200
    # greedy longest-match-first algorithm to perform tokenization
    # using the given vocabulary
    tokens = convert_to_unicode(tokens)
    output_tokens_ = []
    # fn: whitespace_tokenize
    tokens = (' '.join(tokens)).strip()
    if not tokens: tokens = []
    else: tokens = tokens.split()
    # start lmf algorithm!
    print(tokens)
    for token in tokens:
        chars = list(token)
        if len(chars) > max_input_chars_per_word: # 200
            output_tokens_.append(unk_token)
            continue
        is_bad = False
        start = 0
        sub_tokens = []
        while start < len(chars):
            end = len(chars)
            cur_substr = None
            while start < end:
                substr = "".join(chars[start:end])
#                 print(substr)
                if substr in vocab:
                    cur_substr = substr
                    break
                end -= 1
            if cur_substr is None:
                is_bad = True
                break
            sub_tokens.append(cur_substr)
            start = end
        if is_bad:
            output_tokens_.append(unk_token)
        else:
            output_tokens_.extend(sub_tokens)
    for sub_token in output_tokens_:
        split_tokens.append(sub_token)
print('\nsplit_tokens : ', split_tokens)

************** START TOKENING MORPHLOGY **************

Origin Text:   (/SSO XXX/SL 고객/NNG 님/XSN 안녕/NNG 하/XSV 세요/EP+EF 유익/XR 한/XSA+ETM 어/IC 플/NNG 세/MM 가지/NNBC 소개/NNG 해/XSV+EC 드리/VX 고자/EC 합니다/VX+EF ./SF 1/SN ./SF 스타/NNG 알림/VV+ETN :/SC 6/SN 개/NNBC 의/JKG 비밀/NNG 번호/NNG 만/JX 으로/JKB 거래/NNG 내역/NNG 조회/NNG 가능/NNG 입출금/NNG 통지/NNG 가/JKS 무료/NNG (/SSO 창구/NNG 에서/JKB 는/JX 한/MM 달/NNG 에/JKB 900/SN 원/NNBC or/SL 한/MM 건/NNBC 에/JKB 20/SN 원/NNBC )/SSC 2/SN ./SF 리브/NNG :/SC USJPEU/SL 환전/NNG 우대/NNG 90/SN %/SY 번호표/NNG 미리/MAG 뽑/VV 기/ETN 지인/NNG 에게/JKB 로그인/NNG 없이/MAG 바로/MAG 이체/NNG 창구/NNG 에서/JKB 통장/NNG 없이/MAG 바로/MAG 출금/NNG 송금/NNG 3/SN ./SF 리브/NNG 메이트/NNG :/SC 잠자/VV 고/EC 있/VX 는/ETM 카드/NNG 포/NNG 인/VCP+ETM 트리/NNG 조회/NNG 계좌/NNG 로/JKB 현금화/NNG 송금/NNG 어/IC 플/NNG 설치/VV 시/EP 직원/NNG 번호/NNG XXX/SL 꼭/MAG 넣/VV 어/EC 주/VX 세요/EP+EF ./SF 언제나/MAG XXX/SL 은행/NNG 을/JKO 이용/NNG 해/XSV+EC 주/VX 셔서/EP+EC 감사/NNG 합니다/XSV+EF ./SF XXX/SL 은행/NNG 판교/NNG 종합/NNG 금융/NNG 센터/NNG XXX/SL 올림/VV+EC XXX/SL -/SY XXX/SL -/SY XXX/SL 무료/NNG 수신/NNG 거부/NNG XXX/SL

# Wordpiece는 아닌듯싶다...

In [27]:
tokens_a = example.text_a.split(' ')

In [28]:
max_seq_length = 500

In [29]:
print(tokens_a)
if len(tokens_a) > max_seq_length - 2:
    tokens_a = tokens_a[:(max_seq_length - 2)]
print(tokens_a)

['(/SSO', 'XXX/SL', '고객/NNG', '님/XSN', '안녕/NNG', '하/XSV', '세요/EP+EF', '유익/XR', '한/XSA+ETM', '어/IC', '플/NNG', '세/MM', '가지/NNBC', '소개/NNG', '해/XSV+EC', '드리/VX', '고자/EC', '합니다/VX+EF', './SF', '1/SN', './SF', '스타/NNG', '알림/VV+ETN', ':/SC', '6/SN', '개/NNBC', '의/JKG', '비밀/NNG', '번호/NNG', '만/JX', '으로/JKB', '거래/NNG', '내역/NNG', '조회/NNG', '가능/NNG', '입출금/NNG', '통지/NNG', '가/JKS', '무료/NNG', '(/SSO', '창구/NNG', '에서/JKB', '는/JX', '한/MM', '달/NNG', '에/JKB', '900/SN', '원/NNBC', 'or/SL', '한/MM', '건/NNBC', '에/JKB', '20/SN', '원/NNBC', ')/SSC', '2/SN', './SF', '리브/NNG', ':/SC', 'USJPEU/SL', '환전/NNG', '우대/NNG', '90/SN', '%/SY', '번호표/NNG', '미리/MAG', '뽑/VV', '기/ETN', '지인/NNG', '에게/JKB', '로그인/NNG', '없이/MAG', '바로/MAG', '이체/NNG', '창구/NNG', '에서/JKB', '통장/NNG', '없이/MAG', '바로/MAG', '출금/NNG', '송금/NNG', '3/SN', './SF', '리브/NNG', '메이트/NNG', ':/SC', '잠자/VV', '고/EC', '있/VX', '는/ETM', '카드/NNG', '포/NNG', '인/VCP+ETM', '트리/NNG', '조회/NNG', '계좌/NNG', '로/JKB', '현금화/NNG', '송금/NNG', '어/IC', '플/NNG', '설치/VV', '시/EP', '직원/NNG', '번호/

In [30]:
tokens = []
segment_ids = []
tokens.append("[CLS]")
segment_ids.append(0)
for token in tokens_a:
    tokens.append(token)
    segment_ids.append(0)
tokens.append("[SEP]")
segment_ids.append(0)

In [32]:
segment_ids

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]